# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

dummy_list = ["loan_status","pymnt_plan","hardship_flag", "debt_settlement_flag","home_ownership", "verification_status","issue_d", "initial_list_status","next_pymnt_d","application_type"] 


df = pd.get_dummies(df,columns = dummy_list ,drop_first = True)

# Preview the data

df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,0,0,0,1,1,1,0


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns = "loan_status_low_risk")

# Create our target
y = df[["loan_status_low_risk"]]


In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
# YOUR CODE HERE

Counter(y['loan_status_low_risk'])

Counter({1: 68470, 0: 347})

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
# YOUR CODE HERE

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()



In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE

X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
# YOUR CODE HERE

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)




## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

y_pred_rf = brf.predict(X_test_scaled)

In [13]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred_rf)

0.7757796798155026

In [14]:
# Display the confusion matrix
# YOUR CODE HERE

confusion_matrix(y_test, y_pred_rf)

array([[   66,    35],
       [ 1743, 15361]], dtype=int64)

In [15]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.04      0.65      0.90      0.07      0.77      0.57       101
          1       1.00      0.90      0.65      0.95      0.77      0.60     17104

avg / total       0.99      0.90      0.65      0.94      0.77      0.60     17205



In [16]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
# https://stackoverflow.com/questions/42128545/how-to-print-the-order-of-important-features-in-random-forest-regression-using-p
# get importance
cols = df.columns

importance = brf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: {%0d}, Score: %.5f' % (i,v))

Feature: {0}, Score: 0.01205
Feature: {1}, Score: 0.03335
Feature: {2}, Score: 0.01715
Feature: {3}, Score: 0.01443
Feature: {4}, Score: 0.01740
Feature: {5}, Score: 0.00408
Feature: {6}, Score: 0.00480
Feature: {7}, Score: 0.00812
Feature: {8}, Score: 0.00149
Feature: {9}, Score: 0.01592
Feature: {10}, Score: 0.01060
Feature: {11}, Score: 0.01546
Feature: {12}, Score: 0.01595
Feature: {13}, Score: 0.05745
Feature: {14}, Score: 0.06245
Feature: {15}, Score: 0.07238
Feature: {16}, Score: 0.05773
Feature: {17}, Score: 0.00793
Feature: {18}, Score: 0.00000
Feature: {19}, Score: 0.00000
Feature: {20}, Score: 0.05966
Feature: {21}, Score: 0.00035
Feature: {22}, Score: 0.00000
Feature: {23}, Score: 0.00000
Feature: {24}, Score: 0.00364
Feature: {25}, Score: 0.01396
Feature: {26}, Score: 0.00659
Feature: {27}, Score: 0.00574
Feature: {28}, Score: 0.00478
Feature: {29}, Score: 0.00833
Feature: {30}, Score: 0.01476
Feature: {31}, Score: 0.01374
Feature: {32}, Score: 0.01431
Feature: {33}, Score

### Easy Ensemble Classifier

In [17]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier

erf = EasyEnsembleClassifier(n_estimators=1000, random_state=1)

erf.fit(X_train_scaled, y_train)

y_pred_ef = erf.predict(X_test_scaled)



In [18]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

balanced_accuracy_score(y_test, y_pred_ef)

0.9334725129435302

In [19]:
# Display the confusion matrix
# YOUR CODE HERE

confusion_matrix(y_test, y_pred_ef)

array([[   93,     8],
       [  921, 16183]], dtype=int64)

In [20]:
# Print the imbalanced classification report
# YOUR CODE HERE

print(classification_report_imbalanced(y_test, y_pred_ef))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.95      0.17      0.93      0.87       101
          1       1.00      0.95      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.95      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The easy ensemble classifier.

2. Which model had the best recall score?

    The easy ensemble classifier.

3. Which model had the best geometric mean score?

    The easy ensemble classifier.

4. What are the top three features?

    Feature 13, 14, and 15. 